In [9]:
import tensorflow as tf

# creates nodes in a graph
# "construction phase"
x1 = tf.constant(5)
x2 = tf.constant(6)

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
result = x1*x2
print(result)

Tensor("mul:0", shape=(), dtype=int32)


In [13]:
sess = tf.Session()
print(sess.run(result))

30


In [14]:
with tf.Session() as sess:
    output = sess.run(result)
    print(output)

30


In [40]:
with tf.Session() as sess:
    with tf.device("/cpu:1"):
        matrix1 = tf.constant([[3.,3.]])
        matrix2 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix1, matrix2)
        # print(sess.run(product))

In [41]:
print(product)

Tensor("MatMul_21:0", shape=(1, 1), dtype=float32, device=/device:CPU:1)


In [42]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data", one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [43]:
#mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x1c29ae2cc0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x1c29f11fd0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x1c33867908>)

In [85]:
from create_sentiment_featuresets import create_feature_sets_and_labels

In [86]:
train_x, train_y, test_x, test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')

423


In [87]:
# n_nodes_hl1 = 500
# n_nodes_hl2= 500
# n_nodes_hl3 = 500
# n_classes = 10
# batch_size = 100

# for sentiment analysis
n_nodes_hl1 = 1500
n_nodes_hl2 = 1500
n_nodes_hl3 = 1500

n_classes = 2
batch_size = 100
hm_epochs = 10

In [88]:
# x = tf.placeholder('float', [None, 784])
# y = tf.placeholder('float')

x = tf.placeholder('float')
y = tf.placeholder('float')

In [89]:
hidden_1_layer = {'weights': tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                             'biases': tf.Variable(tf.random_normal([n_nodes_hl1])) }
hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                             'biases': tf.Variable(tf.random_normal([n_nodes_hl2])) }
hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                             'biases': tf.Variable(tf.random_normal([n_nodes_hl3])) }
output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                           'biases': tf.Variable(tf.random_normal([n_classes])) }

def neural_network_model(data):
    
    l1 =  tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    
    return output

In [91]:
# print(neural_network_model(mnist.test.images))

In [92]:
import numpy as np
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            # for mnist
#             while i < len(train)
#             for _ in range(int(mnist.train.num_examples/batch_size)):
#                 epoch_x, epoch_y = mnist.train.next_batch(batch_size)
#                 _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
#                 epoch_loss += c
            i = 0
            while i < len(train_x):
                start = i
                end = i + batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y:batch_y})
                
                epoch_loss += c
                i += batch_size
                
            print('Epoch', epoch+1, 'completed out of', hm_epochs, 'loss:',  epoch_loss)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        # print('Accuracy:', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        print('Accuracy:', accuracy.eval({x:test_x, y:test_y}))

In [93]:
# for mnist
train_neural_network(x)

Epoch 1 completed out of 10 loss: 1296028.4169921875
Epoch 2 completed out of 10 loss: 476142.181640625
Epoch 3 completed out of 10 loss: 355693.32751464844
Epoch 4 completed out of 10 loss: 357884.5509338379
Epoch 5 completed out of 10 loss: 274738.75497436523
Epoch 6 completed out of 10 loss: 119955.82341384888
Epoch 7 completed out of 10 loss: 53544.397468566895
Epoch 8 completed out of 10 loss: 43177.28339910507
Epoch 9 completed out of 10 loss: 35468.334550857544
Epoch 10 completed out of 10 loss: 21210.453884124756
Accuracy: 0.5919325


In [56]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [60]:
!mv create_sentiment_featuresets.py.html create_sentiment_featuresets.py

In [61]:
!cat create_sentiment_featuresets.py


# coding: utf-8

# In[2]:


import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import random
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
hm_lines = 100000


# In[3]:


def create_lexicon(pos, neg):
    lexicon = []
    with open(pos, 'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            all_words = word_tokenize(l)
            lexicon += list(all_words)
            
    with open(neg, 'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            all_words = word_tokenize(l)
            lexicon += list(all_words)
            
    lexicon = [lemmatizer.lemmatize(i) for i in lexicon]
    w_counts = Counter(lexicon)
    l2 = []
    for w in w_counts:
        if 1000 > w_counts[w] > 50:
            l2.append(w)
    print(len(l2))
    return l2


# In[7]:


def sample_handling(sample, lexicon, classification):
    featureset 